In [4]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.text import Tokenizer as tk
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import csv

In [3]:
df_train = pd.read_csv('./YELP_train.csv', encoding = "ISO-8859-1")
df_test = pd.read_csv('./YELP_test.csv')
df_val = pd.read_csv('./YELP_val.csv')

In [4]:
df_train.head()

,review,label
0,Excellent studio! Jodi is an amazing instructo...,2
1,Delycios,2
2,I used to like this place despite the fact tha...,1
3,I had the whole house done last April. They d...,2
4,"I know, I know, another rave review for St. Fr...",2


In [5]:
df_train["label"].value_counts()

1    52561
2    52439
Name: label, dtype: int64

In [7]:
X_train = np.array(df_train.drop(columns=['label'])).reshape(len(df_train),)
y_train = np.array(df_train['label'])

X_test = np.array(df_test.drop(columns=['label'])).reshape(len(df_test),)
y_test = np.array(df_test['label'])

X_val = np.array(df_val.drop(columns=['label'])).reshape(len(df_val),)
y_val = np.array(df_val['label'])

In [8]:
xtr = X_train
ytr = y_train

xte = X_test
yte = y_test

xval = X_val
yval = y_val

In [9]:
tok=tk(num_words=30522, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=' ',char_level=False, oov_token='Farid')

In [10]:
tok.fit_on_texts(xtr)

In [11]:
xtr1 = tok.texts_to_sequences(xtr)
xte1 = tok.texts_to_sequences(xte)
xval1 = tok.texts_to_sequences(xval)

In [12]:
sequence_len = 400
xtr2 = pad_sequences(xtr1,sequence_len, padding='post',truncating='post')
xte2 = pad_sequences(xte1,sequence_len, padding='post', truncating='post')
xval2 = pad_sequences(xval1,sequence_len, padding='post', truncating='post')

In [13]:
xtr2

array([[  334,  2262, 18533, ...,     0,     0,     0],
       [    1,     0,     0, ...,     0,     0,     0],
       [    4,   301,     5, ...,     0,     0,     0],
       ...,
       [   17,    32,    12, ...,     0,     0,     0],
       [   56,   120,   735, ...,     3,     2,   182],
       [ 1278,   127,    14, ...,     0,     0,     0]], dtype=int32)

In [14]:
xtr3 = np.flip(xtr2, 1)
xte3 = np.flip(xte2, 1)
xval3 = np.flip(xval2, 1)

In [15]:
print(xtr3.shape)
print(xte3.shape)
print(xval3.shape)

(105000, 400)
(22500, 400)
(22500, 400)


## Training base model

In [ ]:
embedding_vecor_length = 768
max_review_length = 400
NUM_WORDS = 30522

tf.keras.backend.clear_session()
model = Sequential()
model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(GRU(50, return_sequences=True))
# model.add(GRU(50))
model.add(Dropout(0.5))
model.add(Dense(100,activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 50)           22800     
                                                                 
 dropout (Dropout)           (None, 200, 50)           0         
                                                                 
 dense (Dense)               (None, 200, 100)          5100      
                                                        

In [ ]:
class TimeForEpoch(Callback):
    def on_train_begin(self, logs={}):
        self.times=[]
        
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = time.time()
        
    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time()-self.starttime)

cb = TimeForEpoch()

In [ ]:
history = model.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 3699.216743707657


In [2]:
starttime = time.time()
print("Test accuracy: ", np.round(model.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy: 86.058 %


In [3]:
starttime = time.time()
print("Validation accuracy: ", np.round(model.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy: 87.394 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.004716730308532715


## Tuning number of hidden neurons

### 100 neurons in Dense layer

In [5]:
def make_model(neurons):
  embedding_vecor_length = 768
  max_review_length = 400
  NUM_WORDS = 30522

  tf.keras.backend.clear_session()
  model = Sequential()
  model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
  model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D())
  model.add(GRU(neurons, return_sequences=True))
  # model.add(GRU(50))
  model.add(Dropout(0.5))
  model.add(Dense(100,activation='relu'))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  return model

In [6]:
model_100_neurons = make_model(100)
model_100_neurons.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model_100_neurons.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 100)          60600     
                                                                 
 dropout (Dropout)           (None, 200, 100)          0         
                                                                 
 dense (Dense)               (None, 200, 100)          10100     
                                                        

In [ ]:
history = model_100_neurons.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [7]:
average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 3856.3702076673508


In [8]:
starttime = time.time()
print("Test accuracy: ", np.round(model_100_neurons.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy: 86.867 %


In [9]:
starttime = time.time()
print("Validation accuracy: ", np.round(model_100_neurons.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy: 87.345 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.001367732071876526


### 200 neurons in Dense layer

In [10]:
model_200_neurons = make_model(200)
model_200_neurons.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model_200_neurons.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 200)          181200    
                                                                 
 dropout (Dropout)           (None, 200, 200)          0         
                                                                 
 dense (Dense)               (None, 200, 100)          20100     
                                                        

In [ ]:
history = model_200_neurons.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [11]:
average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 3975.7478427886963


In [12]:
starttime = time.time()
print("Test accuracy: ", np.round(model_200_neurons.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy: 85.176 %


In [13]:
starttime = time.time()
print("Validation accuracy: ", np.round(model_200_neurons.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy: 84.972 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.002366953635215759


## Tuning number of layers

### 2 hidden layers

In [14]:
def make_layer_model(layer):
  embedding_vecor_length = 768
  max_review_length = 400
  NUM_WORDS = 30522

  tf.keras.backend.clear_session()
  model = Sequential()
  model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
  model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D())
  model.add(GRU(100, return_sequences=True))
  for i in range(layer):
    model.add(GRU(50))
  model.add(Dropout(0.5))
  # for i in range(layer):
  model.add(Dense(100,activation='relu'))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  return model

In [15]:
model_2_layers = make_layer_model(1)
model_2_layers.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model_2_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 200, 100)          60600     
                                                                 
 gru_1 (GRU)                 (None, 50)                22800     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                        

In [ ]:
history = model_2_layers.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [16]:
average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 3794.742085456848


In [17]:
starttime = time.time()
print("Test accuracy: ", np.round(model_2_layers.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Test accuracy: 86.121 %


In [18]:
starttime = time.time()
print("Validation accuracy: ", np.round(model_2_layers.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy: 86.604 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.0013858426809310912


## Keras LSTM with attention

In [19]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [20]:
def myAct(out):
    return K.softmax(K.tanh(out))

embedding_vecor_length = 768
max_review_length = 400
NUM_WORDS = 30522

tf.keras.backend.clear_session()
model = Sequential()
model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(GRU(50, return_sequences=False, activation=myAct))
# model.add(GRU(50))
model.add(Dropout(0.5))
model.add(Dense(100,activation='relu'))

# model.add(attention())
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23440896  
                                                                 
 conv1d (Conv1D)             (None, 400, 100)          230500    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 200, 100)         0         
 )                                                               
                                                                 
 gru (GRU)                   (None, 50)                22800     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 100)               5100      
                                                        

In [19]:
class TimeForEpoch(Callback):
    def on_train_begin(self, logs={}):
        self.times=[]
        
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = time.time()
        
    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time()-self.starttime)

cb = TimeForEpoch()

In [1]:
history = model.fit(xtr3,ytr,epochs=2,batch_size=16,validation_data = (xval3,yval),shuffle=True,callbacks=[cb])

In [22]:
def Average(lst):
    return sum(lst) / len(lst)

average_epoch_time = Average(cb.times)
print("Average epoch time:", average_epoch_time)

Average epoch time: 3422.4526838064194


In [23]:
starttime = time.time()
print("Accuracy: ", np.round(model.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Accuracy: 87.212 %


In [24]:
starttime = time.time()
print("Validation accuracy: ", np.round(model.evaluate(xval3,yval,verbose=0)[1]*100,3),"%")
endtime = time.time() - starttime

Validation accuracy: 87.722 %


In [ ]:
print("Average time per prediction:", endtime/len(xte3))

Average time per prediction: 0.001035772466659546
